In [ ]:
%load_ext autoreload
%autoreload 2

In [78]:
from constants.companies import get_company_by_code
company = get_company_by_code("GL")
company_id = company.company_id

In [ ]:
from tofu.data import get_order_history, get_calendar

df_order_history = get_order_history(
    company_id=company_id
)

df_calendar = get_calendar()

In [80]:
from tofu.analysis.features import add_holiday_features
df_order_history_with_holidays = add_holiday_features(df_order_history, country="Norway")

In [81]:
import pandas as pd

df_manual_forecast = pd.read_csv(f"./projects/tofu/test_data/df_analysts_predictions_{company.company_code}.csv")
df_manual_forecast = df_manual_forecast.drop_duplicates(subset=["year", "week","variation_id", "prediction_date"])
df_manual_forecast_agged = pd.DataFrame(df_manual_forecast.groupby(["year", "week","prediction_date"])["quantity"].sum()).reset_index()
df_manual_forecast_agged = df_manual_forecast_agged.rename(
    columns={"year": "menu_year", "week": "menu_week"},
)
df_manual_forecast_agged = df_manual_forecast_agged.merge(df_calendar)
df_manual_forecast_agged["num_days_before_menu_monday"] = (pd.to_datetime(df_manual_forecast_agged["monday_date"]) - pd.to_datetime(df_manual_forecast_agged["prediction_date"])).dt.days
df_manual_forecast_agged["num_weeks_before_menu_monday"] = df_manual_forecast_agged["num_days_before_menu_monday"] // 7

In [ ]:
horizon_index_to_analyze = 6

df_manual_to_analyze = df_manual_forecast_agged[df_manual_forecast_agged["num_weeks_before_menu_monday"] == horizon_index_to_analyze]
df_manual_to_analyze = df_manual_to_analyze.drop_duplicates(subset=["menu_year", "menu_week"])
df_manual_to_analyze_merged = df_manual_to_analyze.merge(df_order_history, on=["menu_year", "menu_week"])
df_manual_to_analyze_merged["error"] = df_manual_to_analyze_merged["quantity"] - df_manual_to_analyze_merged["total_orders"]
df_manual_to_analyze_merged["abs_error"] = abs(df_manual_to_analyze_merged["error"])
df_manual_to_analyze_merged["abs_pct_error"] = abs(df_manual_to_analyze_merged["error"]) / df_manual_to_analyze_merged["total_orders"]

df_manual_to_analyze_merged.head()

In [ ]:
import numpy as np
df_tofu_list = []
for i in np.arange(25) + 1:
    df_tofu = pd.read_csv(f"./projects/tofu/test_data/tofu/{company.company_code}/default_predictions_{i}.csv")
    df_tofu_list.append(df_tofu)

df_tofu_all = pd.concat(df_tofu_list)
df_tofu_all.head()

In [101]:
df_tofu_to_analyze = df_tofu_all[df_tofu_all["horizon_index"] == (horizon_index_to_analyze)].drop(columns=["total_orders"])
df_tofu_to_analyze_merged = df_tofu_to_analyze.merge(df_order_history, on=["menu_year", "menu_week"])
df_tofu_to_analyze_merged["error"] = df_tofu_to_analyze_merged["analysts_prediction"] - df_tofu_to_analyze_merged["total_orders"]
df_tofu_to_analyze_merged["abs_error"] = abs(df_tofu_to_analyze_merged["error"])
df_tofu_to_analyze_merged["abs_pct_error"] = abs(df_tofu_to_analyze_merged["error"]) / df_tofu_to_analyze_merged["total_orders"]

In [102]:
df_ml_outputs = pd.read_csv(
    f"./projects/tofu/test_data/df_ml_outputs_{company.company_code}.csv"
)

df_ml_outputs = df_ml_outputs.rename(
    columns={"year": "menu_year", "week": "menu_week"},
).merge(
    df_calendar,
    on=["menu_year", "menu_week"],
    how="left",
)

df_ml_outputs["num_days_before_menu_monday"] = (pd.to_datetime(df_ml_outputs["monday_date"]) - pd.to_datetime(df_ml_outputs["prediction_date"])).dt.days
df_ml_outputs["num_weeks_before_menu_monday"] = df_ml_outputs["num_days_before_menu_monday"]//7


In [103]:
df_ml_outputs_merged = df_ml_outputs.merge(
    df_order_history,
    on=["menu_year", "menu_week"],
    how="left",
)

df_ml_outputs_merged_to_analyze = df_ml_outputs_merged[df_ml_outputs_merged["num_weeks_before_menu_monday"] == horizon_index_to_analyze]
df_ml_outputs_merged_to_analyze = df_ml_outputs_merged_to_analyze.sort_values(by=["menu_year", "menu_week"])
df_ml_outputs_merged_to_analyze["error"] = df_ml_outputs_merged_to_analyze["num_total_orders"] - df_ml_outputs_merged_to_analyze["total_orders"]
df_ml_outputs_merged_to_analyze["abs_error"] = abs(df_ml_outputs_merged_to_analyze["error"])
df_ml_outputs_merged_to_analyze["abs_pct_error"] = abs(df_ml_outputs_merged_to_analyze["error"]) / df_ml_outputs_merged_to_analyze["total_orders"]

In [ ]:
df_ml_outputs_merged_to_analyze[["menu_year", "menu_week", "num_total_orders", "total_orders", "error", "abs_error", "abs_pct_error"]].sort_values(by=["menu_year", "menu_week"])

In [ ]:
import plotly.graph_objects as go

df_order_history_with_holidays = df_order_history_with_holidays.sort_values(by=["menu_year", "menu_week"])
df_history_to_plot = df_order_history_with_holidays[df_order_history_with_holidays["menu_year"] == 2025]
fig_order = go.Figure()

fig_order.add_trace(
    go.Scatter(
        x=df_history_to_plot["menu_week"],
        y=df_history_to_plot["total_orders"],
        mode="lines+markers",
        marker_size=df_history_to_plot["num_holidays"] * 7 + 5,
        name=f"total orders 2025 - actual",
        text=df_history_to_plot["holiday_list"],
        hovertemplate="week: %{x}<br>total orders: %{y}<br>holidays: %{text}",
    )
)

fig_order.add_trace(
    go.Scatter(
        x=df_manual_to_analyze_merged["menu_week"],
        y=df_manual_to_analyze_merged["quantity"],
        mode="lines+markers",
        # marker_size=df_history_to_plot["num_holidays"] * 7 + 5,
        name=f"analysts prediction, {horizon_index_to_analyze} weeks before menu monday",
        # text=df_history_to_plot["holiday_list"],
        # hovertemplate="week: %{x}<br>total orders: %{y}<br>holidays: %{text}",
    )
)

fig_order.add_trace(
    go.Scatter(
        x=df_tofu_to_analyze_merged["menu_week"],
        y=df_tofu_to_analyze_merged["analysts_prediction"],
        mode="lines+markers",
        # marker_size=df_history_to_plot["num_holidays"] * 7 + 5,
        name=f"tofu prediction, {horizon_index_to_analyze} weeks before menu monday",
        # text=df_history_to_plot["holiday_list"],
        # hovertemplate="week: %{x}<br>total orders: %{y}<br>holidays: %{text}",
    )
)

fig_order.add_trace(
    go.Scatter(
        x=df_ml_outputs_merged_to_analyze["menu_week"],
        y=df_ml_outputs_merged_to_analyze["num_total_orders"],
        mode="lines+markers",
        # marker_size=df_history_to_plot["num_holidays"] * 7 + 5,
        name=f"ml prediction, {horizon_index_to_analyze} weeks before menu monday",
        # text=df_history_to_plot["holiday_list"],
        # hovertemplate="week: %{x}<br>total orders: %{y}<br>holidays: %{text}",
    )
)

In [ ]:
df_tofu_to_analyze_merged["abs_error"].describe()

In [ ]:
df_ml_outputs_merged_to_analyze["abs_error"].describe()

In [ ]:
df_manual_to_analyze_merged["abs_error"].describe()

In [ ]:
import plotly.graph_objects as go

df_order_history_with_holidays = df_order_history_with_holidays.sort_values(by=["menu_year", "menu_week"])
df_history_to_plot = df_order_history_with_holidays[df_order_history_with_holidays["menu_year"] == 2025]
fig_error = go.Figure()

fig_error.add_trace(
    go.Scatter(
        x=df_manual_to_analyze_merged["menu_week"],
        y=df_manual_to_analyze_merged["error"],
        mode="lines+markers",
        # marker_size=df_history_to_plot["num_holidays"] * 7 + 5,
        name=f"analysts prediction, {horizon_index_to_analyze} weeks before menu monday",
        # text=df_history_to_plot["holiday_list"],
        # hovertemplate="week: %{x}<br>total orders: %{y}<br>holidays: %{text}",
    )
)

fig_error.add_trace(
    go.Scatter(
        x=df_tofu_to_analyze_merged["menu_week"],
        y=df_tofu_to_analyze_merged["error"],
        mode="lines+markers",
        marker_size=df_tofu_to_analyze_merged["num_holidays"] * 7 + 5,
        name=f"tofu prediction, {horizon_index_to_analyze} weeks before menu monday",
        text=df_tofu_to_analyze_merged["holiday_list"],
        hovertemplate="week: %{x}<br>error: %{y}<br>holidays: %{text}",
    )
)

fig_error.add_trace(
    go.Scatter(
        x=df_ml_outputs_merged_to_analyze["menu_week"],
        y=df_ml_outputs_merged_to_analyze["error"],
        mode="lines+markers",
        # marker_size=df_ml_outputs_merged_to_analyze["num_holidays"] * 7 + 5,
        name=f"ml prediction, {horizon_index_to_analyze} weeks before menu monday",
        # text=df_history_to_plot["holiday_list"],
        # hovertemplate="week: %{x}<br>total orders: %{y}<br>holidays: %{text}",
    )
)
fig_error.show()


In [ ]:
df_tofu_to_analyze_merged

In [ ]:
import plotly.graph_objects as go

df_order_history_with_holidays = df_order_history_with_holidays.sort_values(by=["menu_year", "menu_week"])
df_history_to_plot = df_order_history_with_holidays[df_order_history_with_holidays["menu_year"] == 2025]
fig_error = go.Figure()

fig_error.add_trace(
    go.Scatter(
        x=df_manual_to_analyze_merged["menu_week"],
        
        y=df_manual_to_analyze_merged["abs_pct_error"],
        mode="lines+markers",
        # marker_size=df_history_to_plot["num_holidays"] * 7 + 5,
        name=f"analysts prediction, {horizon_index_to_analyze} weeks before menu monday",
        # text=df_history_to_plot["holiday_list"],
        # hovertemplate="week: %{x}<br>total orders: %{y}<br>holidays: %{text}",
    )
)

fig_error.add_trace(
    go.Scatter(
        x=df_tofu_to_analyze_merged["menu_week"],
        y=df_tofu_to_analyze_merged["abs_pct_error"],
        mode="lines+markers",
        marker_size=df_tofu_to_analyze_merged["num_holidays"] * 7 + 5,
        name=f"tofu prediction, {horizon_index_to_analyze} weeks before menu monday",
        text=df_tofu_to_analyze_merged["holiday_list"],
        # hovertemplate="week: %{x}<br>total orders: %{y}<br>holidays: %{text}",
    )
)

fig_error.add_trace(
    go.Scatter(
        x=df_ml_outputs_merged_to_analyze["menu_week"],
        y=df_ml_outputs_merged_to_analyze["abs_pct_error"],
        mode="lines+markers",
        # marker_size=df_tofu_to_analyze_merged["num_holidays"] * 7 + 5,
        name=f"ml prediction, {horizon_index_to_analyze} weeks before menu monday",
        # text=df_history_to_plot["holiday_list"],
        # hovertemplate="week: %{x}<br>total orders: %{y}<br>holidays: %{text}",
    )
)
fig_error.show()


In [ ]:
df_tofu_to_analyze_merged["abs_pct_error"].describe()

In [ ]:
df_manual_to_analyze_merged["abs_pct_error"].describe()

In [ ]:
df_ml_outputs_merged_to_analyze["abs_pct_error"].describe()

In [30]:
df_compare = df_manual_to_analyze_merged[["menu_year", "menu_week", "abs_error", "abs_pct_error"]].rename(
    columns={
        "abs_error": "abs_error_manual",
        "abs_pct_error": "abs_pct_error_manual"
    }
)
df_compare = df_compare.merge(
    df_tofu_to_analyze_merged[["menu_year", "menu_week", "abs_error", "abs_pct_error"]].rename(
        columns={
            "abs_error": "abs_error_tofu",
            "abs_pct_error": "abs_pct_error_tofu"
        }
    ),
    on=["menu_year", "menu_week"],
    how="left",
)

df_compare = df_compare.merge(
    df_ml_outputs_merged_to_analyze[["menu_year", "menu_week", "abs_error", "abs_pct_error"]].rename(
        columns={
            "abs_error": "abs_error_ml",
            "abs_pct_error": "abs_pct_error_ml"
        }
    ),
    on=["menu_year", "menu_week"],
    how="left",
)

# minus means TOFU is better than manual
df_compare["tofu_error_minus_manual"] = df_compare["abs_error_tofu"] - df_compare["abs_error_manual"]
df_compare["tofu_error_minus_ml"] = df_compare["abs_error_tofu"] - df_compare["abs_error_ml"]


# minus means TOFU is better than manual
df_compare["tofu_pct_error_minus_manual"] = df_compare["abs_pct_error_tofu"] - df_compare["abs_pct_error_manual"]
df_compare["tofu_pct_error_minus_ml"] = df_compare["abs_pct_error_tofu"] - df_compare["abs_pct_error_ml"]



In [ ]:
df_tofu_to_analyze_merged["abs_error"].describe()

In [ ]:
df_ml_outputs_merged_to_analyze["abs_error"].describe()